In [ ]:
# TU Q101_A - Toy equity premium tension
# --------------------------------------
# This is a single-cell Colab-style notebook.
# It simulates three tiny "worlds" for the equity premium puzzle and
# computes a scalar tension observable T_premium for each world.
#
# Goals:
# - Show how a very simple consumption-based asset pricing model behaves.
# - Make the "equity premium puzzle" visible as a tension between
#   target premia and what the model can generate with reasonable parameters.
#
# No API key is needed. Everything is fully offline and reproducible.

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# 1. Global configuration
# -----------------------------

# Randomness
GLOBAL_SEED = 101
N_PATHS = 12000  # number of Monte Carlo samples per scenario

# Target information for the toy "data"
TARGET_PREMIUM = 0.06  # 6% long-run equity premium band (stylised fact)
GAMMA_GRID_MIN = 0.5
GAMMA_GRID_MAX = 20.0
GAMMA_GRID_STEPS = 80

# Reasonable risk-aversion band for a representative agent.
# If the model needs gamma far outside this band, we treat it as a puzzle.
GAMMA_PLAUSIBLE_LOW = 1.0
GAMMA_PLAUSIBLE_HIGH = 5.0

# Tension weighting for T_premium
PREMIUM_SCALE = 0.02  # 2% difference in premium = 1 unit of deviation
WEIGHT_PREMIUM = 0.4
WEIGHT_GAMMA = 0.4
WEIGHT_VOL = 0.2


# -----------------------------
# 2. Scenario definitions
# -----------------------------
# Each scenario is a tiny world with:
# - consumption growth process (mu_c, sigma_c)
# - risky payoff process (mu_p, sigma_p)
# - correlation between consumption and risky payoff (rho)
# - discount factor beta
#
# All parameters are intentionally simple and hand-tuned. We only want to
# show relative patterns, not to calibrate real markets.

SCENARIOS = {
    "no_puzzle": {
        "name": "High-volatility world (no puzzle)",
        "description": (
            "Consumption growth is volatile and strongly correlated "
            "with the risky asset. A moderate gamma can support a 6% "
            "equity premium without stress."
        ),
        "mu_c": 0.02,
        "sigma_c": 0.10,
        "mu_p": 0.05,
        "sigma_p": 0.20,
        "rho": 0.8,
        "beta": 0.99,
        "premium_target": TARGET_PREMIUM,
        "vol_target_range": (0.15, 0.30),
    },
    "realistic_puzzle": {
        "name": "Low-vol consumption (classic puzzle)",
        "description": (
            "Consumption growth volatility is low, closer to real data. "
            "We still demand a 6% equity premium. The model tends to "
            "require very large gamma to get close."
        ),
        "mu_c": 0.02,
        "sigma_c": 0.02,
        "mu_p": 0.06,
        "sigma_p": 0.20,
        "rho": 0.4,
        "beta": 0.99,
        "premium_target": TARGET_PREMIUM,
        "vol_target_range": (0.15, 0.25),
    },
    "anemic_asset": {
        "name": "Low-vol risky asset (anemic risk)",
        "description": (
            "Consumption volatility is low and the risky asset itself "
            "is not very volatile. Asking for a 6% premium here is almost "
            "impossible without extreme parameters."
        ),
        "mu_c": 0.02,
        "sigma_c": 0.02,
        "mu_p": 0.03,
        "sigma_p": 0.08,
        "rho": 0.3,
        "beta": 0.99,
        "premium_target": TARGET_PREMIUM,
        "vol_target_range": (0.10, 0.18),
    },
}


# -----------------------------
# 3. Helper functions
# -----------------------------

def generate_joint_shocks(n_paths, rho, rng):
    """
    Generate two correlated standard normal shock series.

    Parameters
    ----------
    n_paths : int
        Number of Monte Carlo draws.
    rho : float
        Correlation between the two series (consumption and risky payoff).
    rng : np.random.Generator
        Random number generator.

    Returns
    -------
    z_c : np.ndarray
        Shocks for consumption.
    z_p : np.ndarray
        Shocks for risky payoff, correlated with z_c.
    """
    z_c = rng.normal(size=n_paths)
    z_2 = rng.normal(size=n_paths)
    z_p = rho * z_c + math.sqrt(max(0.0, 1.0 - rho**2)) * z_2
    return z_c, z_p


def simulate_world_once(scenario_key, params, n_paths, seed_offset=0):
    """
    Simulate one world and sweep over gamma values.

    For each gamma in the grid we compute:
    - stochastic discount factor m_t
    - risk-free rate R_f(gamma)
    - risky asset price P_e(gamma)
    - risky return distribution R_e_t(gamma)
    - implied equity premium premium_model(gamma)

    We then find gamma_star that best matches the target premium
    and compute a scalar tension T_premium.

    Returns
    -------
    summary : dict
        Best-fit information for the scenario (gamma_star, premium, vol, R_f, T_premium).
    curve : dict
        Gamma grid and premium grid for plotting.
    """
    rng = np.random.default_rng(GLOBAL_SEED + seed_offset)

    # Unpack parameters
    mu_c = params["mu_c"]
    sigma_c = params["sigma_c"]
    mu_p = params["mu_p"]
    sigma_p = params["sigma_p"]
    rho = params["rho"]
    beta = params["beta"]
    premium_target = params["premium_target"]
    vol_target_low, vol_target_high = params["vol_target_range"]

    # Generate correlated shocks
    z_c, z_p = generate_joint_shocks(n_paths, rho, rng)

    # Build gross consumption growth and risky payoff
    g = np.exp(mu_c + sigma_c * z_c)        # consumption growth C_{t+1} / C_t
    pay_e = np.exp(mu_p + sigma_p * z_p)    # risky asset payoff next period

    # Prepare gamma grid
    gamma_grid = np.linspace(GAMMA_GRID_MIN, GAMMA_GRID_MAX, GAMMA_GRID_STEPS)

    # Storage for the full premium curve
    premium_grid = []
    vol_grid = []
    rf_grid = []

    # Track the best gamma (smallest premium error)
    best = None

    for gamma in gamma_grid:
        # Stochastic discount factor m_{t+1} = beta * (g_{t+1})^{-gamma}
        m = beta * g**(-gamma)
        em = np.mean(m)

        # Risk-free rate: price of risk-free payoff 1 is E[m], so R_f = 1 / E[m]
        r_f = 1.0 / em

        # Price of risky payoff and implied return distribution
        price_e = np.mean(m * pay_e)
        re_path = pay_e / price_e

        premium_model = float(np.mean(re_path) - r_f)
        vol_model = float(np.std(re_path))

        premium_grid.append(premium_model)
        vol_grid.append(vol_model)
        rf_grid.append(r_f)

        diff = premium_model - premium_target

        if best is None or abs(diff) < abs(best["diff"]):
            best = {
                "scenario_key": scenario_key,
                "gamma_star": float(gamma),
                "premium_model": premium_model,
                "premium_target": float(premium_target),
                "diff": diff,
                "vol": vol_model,
                "R_f": float(r_f),
            }

    # Compute scalar tension T_premium
    # 1) Premium mismatch component (scaled by PREMIUM_SCALE)
    diff_premium = abs(best["diff"]) / PREMIUM_SCALE

    # 2) Gamma penalty: outside [GAMMA_PLAUSIBLE_LOW, GAMMA_PLAUSIBLE_HIGH]
    gamma_star = best["gamma_star"]
    if gamma_star < GAMMA_PLAUSIBLE_LOW:
        gamma_penalty = (GAMMA_PLAUSIBLE_LOW - gamma_star) / GAMMA_PLAUSIBLE_LOW
    elif gamma_star > GAMMA_PLAUSIBLE_HIGH:
        gamma_penalty = (gamma_star - GAMMA_PLAUSIBLE_HIGH) / GAMMA_PLAUSIBLE_HIGH
    else:
        gamma_penalty = 0.0

    # 3) Volatility penalty: outside the target vol band
    vol_star = best["vol"]
    if vol_star < vol_target_low:
        vol_penalty = (vol_target_low - vol_star) / vol_target_low
    elif vol_star > vol_target_high:
        vol_penalty = (vol_star - vol_target_high) / vol_target_high
    else:
        vol_penalty = 0.0

    # Final scalar tension (simple weighted sum)
    t_premium = (
        WEIGHT_PREMIUM * diff_premium
        + WEIGHT_GAMMA * gamma_penalty
        + WEIGHT_VOL * vol_penalty
    )
    best["T_premium"] = float(t_premium)

    curve = {
        "gamma_grid": gamma_grid,
        "premium_grid": np.array(premium_grid),
        "vol_grid": np.array(vol_grid),
        "rf_grid": np.array(rf_grid),
    }

    return best, curve


# -----------------------------
# 4. Run all scenarios
# -----------------------------

summaries = []
curves = {}

for idx, (key, params) in enumerate(SCENARIOS.items()):
    best, curve = simulate_world_once(key, params, n_paths=N_PATHS, seed_offset=idx)
    summaries.append(best)
    curves[key] = curve

# Build a DataFrame for nice printing
summary_df = pd.DataFrame(summaries)

# Add human-readable scenario names
summary_df["scenario_name"] = summary_df["scenario_key"].map(
    {k: v["name"] for k, v in SCENARIOS.items()}
)

# Reorder columns and sort by T_premium (lower = closer to plausible world)
summary_df = summary_df[
    [
        "scenario_key",
        "scenario_name",
        "gamma_star",
        "premium_target",
        "premium_model",
        "diff",
        "vol",
        "R_f",
        "T_premium",
    ]
].sort_values("T_premium")


# -----------------------------
# 5. Print textual summary
# -----------------------------

print("TU Q101_A - Toy equity premium tension")
print("--------------------------------------")
print(
    "This notebook simulates three tiny consumption-based worlds and\n"
    "computes a scalar tension observable T_premium for each world.\n"
)
print("All runs are fully offline. No API key is needed.\n")

print("Target band and plausible parameter region:")
print(f"- Target long-run equity premium (stylised): {TARGET_PREMIUM:.2%}")
print(
    f"- Plausible risk-aversion band for a representative agent: "
    f"gamma in [{GAMMA_PLAUSIBLE_LOW:.1f}, {GAMMA_PLAUSIBLE_HIGH:.1f}]"
)
print(f"- Premium mismatch is scaled by {PREMIUM_SCALE:.2%} per unit.\n")

print("Configured scenarios:")
for key, params in SCENARIOS.items():
    print(f"- {key}: {params['name']}")
    print(f"  {params['description']}")
print()

print("All simulations completed.\n")

print("Summary table (sorted by T_premium, lower means closer to plausible world):")
print(summary_df.to_string(index=False, float_format=lambda x: f"{x:0.4f}"))
print()

# Quick interpretation lines
for _, row in summary_df.iterrows():
    label = row["scenario_key"]
    t_val = row["T_premium"]
    gamma_star = row["gamma_star"]
    prem = row["premium_model"]
    diff = row["diff"]
    rf = row["R_f"]
    vol = row["vol"]

    print(
        f"- {label}: T_premium ≈ {t_val:0.3f}, "
        f"gamma* ≈ {gamma_star:0.1f}, "
        f"premium_model ≈ {prem:0.2%} "
        f"(target {row['premium_target']:0.2%}, diff {diff:0.2%}), "
        f"R_f ≈ {rf:0.3f}, vol ≈ {vol:0.2f}"
    )
print()


# -----------------------------
# 6. Plot: premium vs gamma
# -----------------------------

plt.figure(figsize=(9, 5))

for key, curve in curves.items():
    gamma_grid = curve["gamma_grid"]
    premium_grid = curve["premium_grid"]
    plt.plot(
        gamma_grid,
        premium_grid,
        label=SCENARIOS[key]["name"],
    )

# Target premium band (horizontal band)
plt.axhline(TARGET_PREMIUM, linestyle="--")
plt.axhline(TARGET_PREMIUM - PREMIUM_SCALE, linestyle=":")
plt.axhline(TARGET_PREMIUM + PREMIUM_SCALE, linestyle=":")

# Plausible gamma band (vertical band)
plt.axvspan(
    GAMMA_PLAUSIBLE_LOW,
    GAMMA_PLAUSIBLE_HIGH,
    alpha=0.1,
)

plt.title("TU Q101_A · Model equity premium vs risk aversion gamma")
plt.xlabel("Risk aversion gamma")
plt.ylabel("Model equity premium (E[Re] - Rf)")
plt.legend()
plt.tight_layout()
plt.savefig("Q101A_premium_vs_gamma.png", dpi=150)
plt.show()

print("Saved premium-vs-gamma plot as: Q101A_premium_vs_gamma.png")

# -----------------------------
# 7. Plot: T_premium per scenario
# -----------------------------

plt.figure(figsize=(6, 4))

x_labels = summary_df["scenario_key"].tolist()
x_pos = range(len(x_labels))
t_values = summary_df["T_premium"].values

plt.bar(x_pos, t_values)
plt.xticks(x_pos, x_labels)
plt.ylabel("T_premium (higher = more tension)")
plt.title("TU Q101_A · T_premium per scenario")
plt.tight_layout()
plt.savefig("Q101A_T_premium.png", dpi=150)
plt.show()

print("Saved tension bar plot as: Q101A_T_premium.png")
